In [ ]:
import uproot
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sys
sys.path.append("/scratch/cgarcia_2002/nHits_count")

import functions_nHits  # Importa la función que necesitas
import importlib
import functions_spills

importlib.reload(functions_nHits)
importlib.reload(functions_spills)

%matplotlib inline

In [ ]:
# To load later
with open("filtered_data.pkl", "rb") as f:
    data = pickle.load(f)
    times_branch_modified = data["times_branch_modified"]
    charge_branch_modified = data["charge_branch_modified"]
    times_branch_modified_sig = data["times_branch_modified_sig"]
    charge_branch_modified_sig = data["charge_branch_modified_sig"]

In [ ]:
import csv
from collections import defaultdict

filename = "/scratch/cgarcia_2002/nHits_count/plots_neutronDetection/First_test_total/neutron_candidates.csv"

# Outer dict: key=event_number, value = dict of start_time -> list of neutron_times
data_dict = defaultdict(lambda: defaultdict(list))

with open(filename, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        event = int(row['event_number'])
        start_time = float(row['start_time'])
        neutron_time = float(row['neutron_time'])
        
        # Append neutron_time to the list of neutron times for that start_time in event_number
        data_dict[event][start_time].append(neutron_time)

# To convert defaultdict to normal dict (optional)
neutron_dict = {k: dict(v) for k, v in data_dict.items()}

from pprint import pprint
pprint(neutron_dict)


In [ ]:
neutron_dict_cut = dict(list(neutron_dict.items())[0:10])

print(neutron_dict)

for event_number in neutron_dict_cut:
    for start_time in neutron_dict_cut[event_number]:

        neutron_times = neutron_dict_cut[event_number][start_time]

        bin_edges = np.arange(start_time, start_time + 100001, 100)  # 120 bins

        plt.figure(figsize=(8, 4))
        plt.hist(
            times_branch_modified_chargesTT_sig[event_number],
            bins=bin_edges,
            color='blue',
            edgecolor='navy',
            alpha=0.5,
            label="Total data"
        )
        mask = (times_branch_sup5_sig[event_number] >= start_time) & (times_branch_sup5_sig[event_number] < start_time + 100)
        plt.hist(
            times_branch_sup5_sig[event_number][mask],
            bins=bin_edges,
            histtype='step',
            color='black',
            linewidth=2,
            label="Signal candidate"
        )
        for t in neutron_times:
            mask = (times_branch_modified_chargesTT_sig[event_number] >= t) & (times_branch_modified_chargesTT_sig[event_number] <= t + 100)
            plt.hist(
                times_branch_modified_chargesTT_sig[event_number][mask],
                bins=bin_edges,
                histtype='step',
                color='red',
                linewidth=2,
                label='Neutron time' if t == neutron_times[0] else None
            )
        plt.xlabel('Tiempo (ns)')
        plt.ylabel('Número de hits')
        plt.title(f'evento {event_number}')
        plt.legend()
        plt.show()